In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model, load_model
from keras.callbacks import EarlyStopping
import tensorflow as tf
from keras.utils import plot_model
from scipy.spatial import distance
from scipy.spatial.distance import cdist, cosine
import matplotlib.pyplot as plt
from tensorboard.plugins import projector
import joblib

2023-02-21 00:56:24.190986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 00:56:24.384195: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-21 00:56:25.396200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-21 00:56:25.396466: W tensorfl

In [ ]:
df = pd.read_csv('final_data/fifa_cleaned_all_columns.csv').drop(['Unnamed: 0'], axis=1)
df

,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,body_Stocky,body_Unique,right_foot,left_foot,injury_risk,teamwork,passing_traits,attacking_traits,dribbling_traits,defending_traits
0,158023,L. Messi,CF,93,95,100500000.0,550000.0,27,169,67,...,0,0,3,5,0,1,0,1,1,0
1,20801,Cristiano Ronaldo,"LW, LM",92,92,79000000.0,375000.0,29,185,80,...,0,0,5,4,0,0,0,1,1,0
2,9014,A. Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,...,0,0,2,5,1,-1,0,2,1,0
3,41236,Z. Ibrahimović,ST,90,90,52500000.0,275000.0,32,195,95,...,0,0,5,4,0,1,0,1,1,0
4,41,Iniesta,"CM, LW",89,89,36000000.0,250000.0,30,170,65,...,0,0,5,4,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126283,261962,Song Defu,CDM,47,52,70000.0,1000.0,22,180,64,...,0,0,5,3,0,0,0,0,0,0
126284,262040,C. Porter,CM,47,59,110000.0,500.0,19,175,70,...,0,0,5,3,0,0,0,0,0,0
126285,262760,N. Logue,CM,47,55,100000.0,500.0,21,178,72,...,0,0,5,3,0,0,0,0,0,0
126286,262820,L. Rudden,ST,47,60,110000.0,500.0,19,173,66,...,0,0,5,3,0,0,0,0,0,0


In [ ]:
columns = ['overall', 'potential', 
           #'age', 'height_cm', 'weight_kg', 'league_level', 
           'height_cm', 'weight_kg',
           'skill_moves', #'international_reputation', 
           'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 
           'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 
           'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 
           'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 
           'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 
           'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 
           'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle', 
           'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 
           'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 
           'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 
           #### Traits
           'One Club Player', 'Avoids Using Weaker Foot', 'Playmaker', 'Dives Into Tackles', 'Finesse Shot', 
           'Power Free-Kick', 'Leadership', 'Power Header', 'Technical Dribbler', 'Early Crosser', 'Takes Finesse Free Kicks', 
           'Through Ball', 'Giant Throw-in', 'Beat Offside Trap', 'Outside Foot Shot', 'Long Passer', 'Set Play Specialist', 'Chip Shot', 
           'Diver', 'Team Player', 'Injury Free', 'Injury Prone', 'Swerve Pass', 'Solid Player', 'Selfish', 'Speed Dribbler', 'Flair', 
           'Long Shot Taker', 'Long Throw-in', 'Backs Into Player', 'Target Forward', 
           #### Tags (Remove)
           #'Speedster', 'Complete Defender', 'Dribbler', 
           #'Tackling', 'Acrobat', 'Poacher', 'Crosser', 'FK Specialist', 'Complete Forward', 'Complete Midfielder', 'Engine', 
           #'Clinical Finisher', 'Aerial Threat', 'Tactician', 'Distance Shooter', 'Strength', 
           
           #### Newly created
           'att_workrate', 'def_workrate', 'body_Lean', 'body_Normal', 'body_Stocky', 'body_Unique', 'right_foot', 'left_foot', 
           'injury_risk', 'teamwork', 'passing_traits', 'attacking_traits', 'dribbling_traits', 'defending_traits']

In [ ]:
df[columns]

,overall,potential,height_cm,weight_kg,skill_moves,pace,shooting,passing,dribbling,defending,...,body_Stocky,body_Unique,right_foot,left_foot,injury_risk,teamwork,passing_traits,attacking_traits,dribbling_traits,defending_traits
0,93,95,169,67,4,93.0,89.0,86.0,96.0,27.0,...,0,0,3,5,0,1,0,1,1,0
1,92,92,185,80,5,93.0,93.0,81.0,91.0,32.0,...,0,0,5,4,0,0,0,1,1,0
2,90,90,180,80,4,93.0,86.0,83.0,92.0,32.0,...,0,0,2,5,1,-1,0,2,1,0
3,90,90,195,95,4,76.0,91.0,81.0,86.0,34.0,...,0,0,5,4,0,1,0,1,1,0
4,89,89,170,65,4,75.0,72.0,89.0,91.0,59.0,...,0,0,5,4,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126283,47,52,180,64,2,58.0,35.0,46.0,48.0,42.0,...,0,0,5,3,0,0,0,0,0,0
126284,47,59,175,70,2,59.0,39.0,50.0,46.0,41.0,...,0,0,5,3,0,0,0,0,0,0
126285,47,55,178,72,2,60.0,37.0,45.0,49.0,41.0,...,0,0,5,3,0,0,0,0,0,0
126286,47,60,173,66,2,68.0,46.0,36.0,48.0,15.0,...,0,0,5,3,0,0,0,0,0,0


In [ ]:
### Train test Split
X = df[columns].copy()
# Target
y = df['overall']

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=42)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train.shape

(107344, 111)

In [ ]:
def ae_hpo(emb_dim = 20, epochs=50, batch_size=32):
    input = tf.keras.layers.Input(shape=X_train.shape[1:])

    encoder = tf.keras.layers.Dense(X_train.shape[1:][0], activation='relu')(input)
    encoder = tf.keras.layers.Dense(100, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(75, activation='relu')(encoder)
    encoder = tf.keras.layers.Dense(50, activation='relu')(encoder)
    embedding = tf.keras.layers.Dense(emb_dim, activation='relu',name='embedding')(encoder)

    decoder = tf.keras.layers.Dense(50, activation='relu')(embedding)
    decoder = tf.keras.layers.Dense(75, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(100, activation='relu')(decoder)
    decoder = tf.keras.layers.Dense(X_train.shape[1], activation='sigmoid')(decoder),

    autoencoder = Model(input, decoder)
    autoencoder.compile(loss='mae', optimizer='adam')
    callback = EarlyStopping(monitor='loss', patience=3)
    history = autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, #shuffle=False, 
                callbacks=[callback], validation_data = (X_test, X_test))
    return history

In [ ]:
tune_dim = [10,15,20]
results = []
for dim in tune_dim:
    print('Start Training with Latent Space Dimension: ' + str(dim) + '\n')
    results.append(ae_hpo(dim))
    print('Completed Training with Latent Space Dimension: ' + str(dim) + '\n\n')

Start Training with Latent Space Dimension: 10



2023-02-21 00:56:30.922945: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 00:56:30.934695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 00:56:30.936527: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 00:56:30.939586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/50
3355/3355 [==============================] - 15s 4ms/step - loss: 0.0520 - val_loss: 0.0446
Epoch 2/50
3355/3355 [==============================] - 12s 4ms/step - loss: 0.0387 - val_loss: 0.0370
Epoch 3/50
3355/3355 [==============================] - 13s 4ms/step - loss: 0.0365 - val_loss: 0.0355
Epoch 4/50
3355/3355 [==============================] - 12s 4ms/step - loss: 0.0346 - val_loss: 0.0337
Epoch 5/50
3355/3355 [==============================] - 12s 4ms/step - loss: 0.0331 - val_loss: 0.0330
Epoch 6/50
3355/3355 [==============================] - 13s 4ms/step - loss: 0.0325 - val_loss: 0.0323
Epoch 7/50
3355/3355 [==============================] - 13s 4ms/step - loss: 0.0323 - val_loss: 0.0324
Epoch 8/50
3355/3355 [==============================] - 12s 4ms/step - loss: 0.0322 - val_loss: 0.0328
Epoch 9/50
3355/3355 [==============================] - 12s 4ms/step - loss: 0.0321 - val_loss: 0.0319
Epoch 10/50
3355/3355 [==============================] - 12s 4ms/step - l

In [ ]:
print('For Players Autoencoder Hyperparameter Tuning: ')
print('MAE of Dimension 10: ' + str(min(results[0].history['val_loss'])))
print('MAE of Dimension 15: ' + str(min(results[1].history['val_loss'])))
print('MAE of Dimension 20: ' + str(min(results[2].history['val_loss'])))

For Players Autoencoder Hyperparameter Tuning: 
MAE of Dimension 10: 0.029535505920648575
MAE of Dimension 15: 0.027811067178845406
MAE of Dimension 20: 0.024914314970374107


In [ ]:
player_all_scaled = scaler.transform(X)

svd1 = TruncatedSVD(n_components=10, n_iter=10, random_state=42)
player_embedding_10 = svd1.fit_transform(player_all_scaled)

svd2 = TruncatedSVD(n_components=15, n_iter=10, random_state=42)
player_embedding_15 = svd2.fit_transform(player_all_scaled)

svd3 = TruncatedSVD(n_components=20, n_iter=10, random_state=42)
player_embedding_20 = svd3.fit_transform(player_all_scaled)

In [ ]:
print('For Players SVD Hyperparameter Tuning: ')

player_embedding = pd.DataFrame(player_embedding_10)
a = pd.DataFrame(svd1.inverse_transform(player_embedding))
b = pd.DataFrame(player_all_scaled)
c = abs(a-b).mean().mean()
print('MAE of 10 Components: ' + str(c))

player_embedding = pd.DataFrame(player_embedding_15)
a = pd.DataFrame(svd2.inverse_transform(player_embedding))
#b = pd.DataFrame(player_all_scaled)
c = abs(a-b).mean().mean()
print('MAE of 15 Components: ' + str(c))

player_embedding = pd.DataFrame(player_embedding_20)
a = pd.DataFrame(svd3.inverse_transform(player_embedding))
#b = pd.DataFrame(player_all_scaled)
c = abs(a-b).mean().mean()
print('MAE of 20 Components: ' + str(c))

For Players SVD Hyperparameter Tuning: 
MAE of 10 Components: 0.04721016965871067
MAE of 15 Components: 0.03936541687437166
MAE of 20 Components: 0.03385887335978506
